# 節點部分處理細節---KDNuggets 網頁爬蟲

In [1]:
import requests
import lxml
import pdfkit
import os
from bs4 import BeautifulSoup as bs
from IPython.display import display, HTML

### 讀入網址

In [2]:
url = ['http://www.kdnuggets.com/2017/06/understanding-deep-learning-rethinking-generalization.html',
       'http://www.kdnuggets.com/2017/06/top-15-python-libraries-data-science.html',
       'http://www.kdnuggets.com/2017/06/why-artificial-intelligence-machine-learning.html'
       'http://www.kdnuggets.com/2017/07/when-not-use-deep-learning.html']

### 載入網頁

In [3]:
page = requests.get(url[0])
article = bs(page.text,'lxml')

# 查看網頁原貌(JS、CSS可能影響整個notebook表現
# display(HTML(str(article)))

# 單純查看格式整理後的HTML輸出
# print(str(article.prettify()))

### 細部整理

In [4]:
# 擷取到有全部主內容的最小節點
content = article.find(id='content')

In [5]:
# 去除主內容內的雜訊(e.g.廣告)
content.find(id="post-header-ad").decompose()

# 取出文章部分(此case不包含title，所以後面樣板要加上標題標籤)
mainArt = content.find(id = 'post-')

# 由於此case有一文多頁的狀況 故要取出其他頁的連結部分)
link = content.find(class_ = 'page-link').extract()

In [6]:
# 取出標題部分(部分文章第一個內容物是圖檔(熱門或是精華文章標示)故要擷取最末位(-1))
title = str(content.find(id='title').contents[-1])

filetitle = title[:]
# 暫存HTML和後續PDF的檔名不能包含部分符號之移除
for ch in '*|\\:"<>?/':
    filetitle = filetitle.replace(ch, "")

### 建立新網頁節點樹

In [7]:
# 建立樣板
newPage = bs('<html><head><title></title><meta charset="UTF-8"></head><body><h1></h1></body></html>','lxml')

# 插入網頁標題、標題、文章
newPage.title.insert(len(newPage.title.contents),title)
newPage.h1.insert(len(newPage.h1.contents),title)
newPage.body.insert(len(newPage.body.contents),mainArt)

In [8]:
# 將後段分頁的內容也加入(只需要文章部分，狀況同第一頁的處理)
for subpage in link.find_all('a'):
    Spage = requests.get(subpage['href'])
    Sarticle = bs(Spage.text,'lxml')
    Scontent = Sarticle.find(id='content')
    SmainArt = Scontent.find(id = 'post-')
    newPage.body.insert(len(newPage.body.contents),SmainArt)

### 檔案匯出

In [9]:
#建立HTML檔
with open(filetitle+'.html','w',encoding='utf-8') as file:
    file.write(newPage.prettify())
    
#讀入HTML檔，產生PDF
if os.path.exists(filetitle+'.pdf'):
    os.remove(filetitle+'.pdf')
pdfkit.from_file(filetitle+'.html', filetitle+'.pdf')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


True